<a href="https://colab.research.google.com/github/remicastaing/Colab/blob/main/Copie_de_P3_Distilling_the_Knowledge_distillation_selon_exemple_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install keras --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.5 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.4 which is incompatible.


In [ ]:
data_path="/content/drive/MyDrive/CNAM/RCP209/Projet - Distilling the Knowledge in a Neural Network/data/"

images_path="/content/drive/MyDrive/CNAM/RCP209/Projet - Distilling the Knowledge in a Neural Network/Rapport/Images/"

resnet50_model_file = "resnet50-model-keras3.keras"

resnet18_model_file = "ResNet18-81.keras"

input_shape = (32, 32, 3)
classes = 10

#1. Import des bibliothèque

In [ ]:
import keras

from keras.applications.resnet50 import ResNet50, preprocess_input

from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics

from keras import Model
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import AveragePooling2D
from keras.layers import Flatten
from keras.layers import MaxPool2D
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Lambda
from keras.layers import concatenate
from keras.regularizers import l2



from keras.utils import to_categorical
from keras.models import load_model
from keras.datasets import cifar10
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import pickle

#2. Definition fonction graphique

In [ ]:
def loss_accuracy_graph(history, nom_fichier):

  history_dict = history.history
  loss_values = history_dict['loss']
  val_loss_values = history_dict['val_loss']

  epochs = range(1, len(loss_values) + 1)

  fig = plt.figure(figsize=(14, 4))

  plt.subplot(1,2,1)
  plt.plot(epochs, loss_values, 'bo', label="Entrainement")
  plt.plot(epochs, val_loss_values, 'b', label='Validation')
  plt.title("Coût à l'entrainement et à la validation")
  plt.xlabel('Epochs')
  plt.ylabel('Coût')
  plt.legend()

  acc = history_dict['accuracy']
  val_acc = history_dict['val_accuracy']

  epochs = range(1, len(loss_values) + 1)

  plt.subplot(1,2,2)
  plt.plot(epochs, acc, 'o', label="Entrainement", c='orange')
  plt.plot(epochs, val_acc, '-', label='Validation', c='orange')
  plt.title("Précision à l'entrainement et à la validation")
  plt.xlabel('Epochs')
  plt.ylabel('Précision')
  plt.legend()

  plt.savefig(images_path + nom_fichier + '.png')

  with open(images_path + nom_fichier + '_hist', 'wb') as file_pi:
    pickle.dump(history_dict, file_pi)

# 3. Import des données

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

print(x_train.shape)
print(x_test.shape)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
(50000, 32, 32, 3)
(10000, 32, 32, 3)


#4. Import et modification du modèle professeur ResNet50

In [ ]:
#import du modèle ResNet50 entrainé précédemment

resnet50_model = keras.models.load_model(data_path + resnet50_model_file)

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:394: UserWarning: Skipping variable loading for optimizer 'adam', because it has 458 variables whereas the saved optimizer has 34 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


#5. Définition du modèle ResNet18

In [ ]:
def conv2d_bn(x, filters, kernel_size, weight_decay=.0, strides=(1, 1)):
    layer = Conv2D(filters=filters,
                   kernel_size=kernel_size,
                   strides=strides,
                   padding='same',
                   use_bias=False,
                   kernel_regularizer=l2(weight_decay)
                   )(x)
    layer = BatchNormalization()(layer)
    return layer


def conv2d_bn_relu(x, filters, kernel_size, weight_decay=.0, strides=(1, 1)):
    layer = conv2d_bn(x, filters, kernel_size, weight_decay, strides)
    layer = Activation('relu')(layer)
    return layer


def ResidualBlock(x, filters, kernel_size, weight_decay, downsample=True):
    if downsample:
        # residual_x = conv2d_bn_relu(x, filters, kernel_size=1, strides=2)
        residual_x = conv2d_bn(x, filters, kernel_size=1, strides=2)
        stride = 2
    else:
        residual_x = x
        stride = 1
    residual = conv2d_bn_relu(x,
                              filters=filters,
                              kernel_size=kernel_size,
                              weight_decay=weight_decay,
                              strides=stride,
                              )
    residual = conv2d_bn(residual,
                         filters=filters,
                         kernel_size=kernel_size,
                         weight_decay=weight_decay,
                         strides=1,
                         )
    out = layers.add([residual_x, residual])
    out = Activation('relu')(out)
    return out



def ResNet18(weight_decay=1e-4):
    input = Input(shape=input_shape)
    x = input
    x = conv2d_bn_relu(x, filters=64, kernel_size=(3, 3), weight_decay=weight_decay, strides=(1, 1))

    # # conv 2
    x = ResidualBlock(x, filters=64, kernel_size=(3, 3), weight_decay=weight_decay, downsample=False)
    x = ResidualBlock(x, filters=64, kernel_size=(3, 3), weight_decay=weight_decay, downsample=False)
    # # conv 3
    x = ResidualBlock(x, filters=128, kernel_size=(3, 3), weight_decay=weight_decay, downsample=True)
    x = ResidualBlock(x, filters=128, kernel_size=(3, 3), weight_decay=weight_decay, downsample=False)
    # # conv 4
    x = ResidualBlock(x, filters=256, kernel_size=(3, 3), weight_decay=weight_decay, downsample=True)
    x = ResidualBlock(x, filters=256, kernel_size=(3, 3), weight_decay=weight_decay, downsample=False)
    # # conv 5
    x = ResidualBlock(x, filters=512, kernel_size=(3, 3), weight_decay=weight_decay, downsample=True)
    x = ResidualBlock(x, filters=512, kernel_size=(3, 3), weight_decay=weight_decay, downsample=False)
    x = AveragePooling2D(pool_size=(4, 4), padding='valid')(x)

    x = Flatten()(x)
    x = Dense(classes, name="last_dense")(x)
    output = Activation('softmax', name="output")(x)


    model = Model(input, output, name='ResNet18_temp')
    return model

#6. Préparation des modèles élève et profsseur

In [ ]:
def preparation_etudiant(model):

  input_tensor = model.input

  output = model.get_layer("output").output

  output_digits = model.get_layer("last_dense").output
#  temperature_layer = Lambda(lambda x: x / temperature, name='temperature_T' + str(temperature))(last_dense)
#  output_tempere = Activation('softmax', name='output_tempere')(temperature_layer)

  etudiant = Model(input_tensor, [output, output_digits], name='ResNet18_temperere')

  return etudiant

In [ ]:
def loss_tempere(temperature) :

    return lambda y_D10_digits, y_pred_digits : losses.categorical_crossentropy(Activation('softmax')(y_D10_digits/temperature), Activation('softmax')(y_pred_digits/temperature))



In [ ]:
def preparation_professeur(model):

  input_tensor = model.input

  last_dense = model.get_layer("last_dense").output

  professeur = Model(input_tensor, last_dense, name='ResNet50_digits')

  return professeur

In [ ]:
professeur = preparation_professeur(resnet50_model)

In [ ]:
digits = professeur.predict(x_train)

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 152s 90ms/step


In [ ]:
weight_decay = 1e-4
temperature = 5
alpha = 0.3
batch_size = 64
epochs = 10

resnet18 = ResNet18( weight_decay=weight_decay)

etudiant = preparation_etudiant(resnet18)

etudiant.compile(optimizer='Adam',
                loss=[losses.categorical_crossentropy, loss_tempere(temperature)],
                loss_weights=[1-alpha, alpha],
                metrics={'output' : ['accuracy'], 'last_dense': [metrics.KLDivergence()]})

history = etudiant.fit(x_train, [y_train, digits],
          batch_size=batch_size,
          epochs=epochs,
          verbose=1)

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 71s 70ms/step - last_dense_kl_divergence: 2.5023 - loss: 2.0807 - output_accuracy: 0.4533
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 48s 61ms/step - last_dense_kl_divergence: 2.3224 - loss: 1.3876 - output_accuracy: 0.7115
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 82s 61ms/step - last_dense_kl_divergence: 3.3375 - loss: 1.2322 - output_accuracy: 0.7747
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 82s 61ms/step - last_dense_kl_divergence: 4.3081 - loss: 1.1540 - output_accuracy: 0.8120
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 82s 61ms/step - last_dense_kl_divergence: 5.2644 - loss: 1.1190 - output_accuracy: 0.8328
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 48s 61ms/step - last_dense_kl_divergence: 6.3896 - loss: 1.0837 - output_accuracy: 0.8555
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 82s 61ms/step - last_dense_kl_divergence: 7.2553 - loss: 1.0415 - output_accuracy: 0.8753
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 82s 61ms/step - last_dense_kl_divergence: 8.1456 -

#7. Preparation données

#8. Entrainement par distillation du savoir

In [ ]:
weight_decay = 1e-4

etudiant = ResNet18( weight_decay=weight_decay)

alembic = Alembic(etudiant, resnet50_model)

alembic.load_data(x_train, x_test, y_train, y_test)

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 45s 26ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step


In [ ]:
batch_size=64
epochs = 100

alembic.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    etudiant_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

alembic.fit(epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/100
(None, 10)
(None, 10)


ValueError: Argument `output` must have rank (ndim) `target.ndim - 1`. Received: target.shape=(None, 10), output.shape=(None, 10)

#9. Evaluation

In [ ]:
def resnet18_sans_temperature(model):
  multiply_layer = Lambda(lambda x: x / temperature)
  softmax_layer = Activation('softmax')

  input_tensor = model.input
  output_tensor = model.get_layer("last_dense").output


  new_output_tensor = Activation('softmax')(output_tensor)

  model = Model(input_tensor, new_output_tensor, name='ResNet18_sans_temp')

  model.compile(optimizer='Adam',
                       loss=losses.categorical_crossentropy,
                       metrics=['accuracy'])

  return model

In [ ]:
resnet18_sans_temperature(resnet18_soft_model).evaluate(x_test, y_test)

313/313 [==============================] - 3s 6ms/step - loss: 1.1237 - accuracy: 0.8369


[1.1237249374389648, 0.836899995803833]

#Test

In [ ]:
restnet50_soft = resnet50_tempere(resnet50_model, 20)

In [ ]:
weight_decay = 1e-4
lr = 1e-1
num_classes = 10
batch_size=256

callback = keras.callbacks.EarlyStopping(monitor='loss', mode='min', min_delta=0.01, baseline=0.2, restore_best_weights=True)

resnet18_model = ResNet18_tempere( weight_decay=weight_decay, temperature=2)
opt = optimizers.SGD(learning_rate=lr, momentum=0.9, nesterov=False)
resnet18_model.compile(optimizer=opt,
                       loss=losses.categorical_crossentropy,
                       metrics=['accuracy'])

history = resnet18_model.fit(x_train, y_train,
                             epochs=100,
                             batch_size=batch_size,
                             validation_data=(x_test, y_test),
                             callbacks=[callback])

Epoch 1/100
 44/196 [=====>........................] - ETA: 55:38 - loss: 2.2331 - accuracy: 0.3231

KeyboardInterrupt: 

In [ ]:
class Alembic(keras.Model):
  def __init__(self, student_model, teacher_model):
    super().__init__()
    self.teacher_model = self.raccourcir(teacher_model)
    self.student_model = self.raccourcir(student_model)

  def raccourcir(self, model):
    input_tensor = model.input
    new_output_tensor = model.get_layer("last_dense").output

    model_raccourci = keras.models.Model(input_tensor, new_output_tensor)

    return model_raccourci

  def distillation_loss_fn(temperature, alpha):

      # Extract the one-hot encoded values and the softs separately so that we can create two objective functions

      def distillation_loss(y_true, y_D10_digits)

        y_true, y_D50_softmax = y_true[: , :classes], y_true[: , classes:]

        y_pred, y_pred_softs = y_pred[: , :classes], y_pred[: , classes:]

        y_pred_soft = Activation('softmax')(y_pred)

        y_D


        output =

        loss = alpha * losses.categorical_crossentropy(y_true,y_pred) + (1 - alpha) * losses.categorical_crossentropy(y_true_softs, y_pred_softs)

      return distillation_loss



  def compile(
      self,
      optimizer,
      metrics,
      student_loss_fn,
      distillation_loss_fn,
      alpha=0.1,
      temperature=3,
      ):

    super().compile(optimizer=optimizer, metrics=metrics)
    self.student_loss_fn = student_loss_fn
    self.distillation_loss_fn = distillation_loss_fn
    self.alpha = alpha
    self.temperature = temperature

  def load_data(self, x_train, x_test, y_train, y_test):



    # Concatenation des données pour obetnir un vecteur de 10 + 10 comprenant la vérité vrai et la prediction du resnet50

    self.x_train = x_train
    self.x_test = x_test

    self.y_train = y_train
    self.y_test = y_test

    self.y_train_digits = self.teacher_model.predict(x_train)
    self.y_test_digits = self.teacher_model.predict(x_test)



  def train(self, batch_size, epochs, verbose, temperature, alpha):

    y_train_distillation = np.concatenate([y_train, self.y_train_digits / self.temperature], axis=1)
    y_test_distillation =  np.concatenate([y_test, self.y_test_digits / self.temperature], axis =1)





    super().fit(self.x_train, self.y_train_distillation,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(self.x_test, self.y_test_distillation))


  def compute_loss(
    self, x=None, y=None, y_pred=None, sample_weight=None, allow_empty=False
    ):

    y, y_softs = y[: , :classes], y[: , classes:]

    print(y.shape)
    print(y_pred.shape)
    etudiant_loss = self.student_loss_fn(y, y_pred)

    distillation_loss = self.distillation_loss_fn(
      keras.ops.softmax(y_softs / self.temperature, axis=1),
      keras.ops.softmax(y_pred / self.temperature, axis=1),
      ) * (self.temperature**2)

    loss = self.alpha * etudiant_loss + (1 - self.alpha) * distillation_loss
    return loss

  def call(self, x):
    return self.etudiant(x)